In [1]:
import pandas as pd
import numpy as np
import time
import torchvision
from torchvision import models
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os
import random
import cv2
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from imgaug import augmenters as iaa
from skimage import io, transform


print(os.listdir("input"))

['test.csv', 'sample_submission.csv', 'train_images.zip', 'test_images.zip', 'train.csv', 'test_images', 'train_images']


In [14]:
model1 = models.densenet121(pretrained=None)
model1.classifier = nn.Linear(1024, 1)
model1.load_state_dict(torch.load("resize_five_crop.bin"))
model1.cuda()

for param in model1.parameters():
    param.requires_grad = False


In [15]:
model2 = models.densenet121(pretrained=None)
model2.classifier = nn.Linear(1024, 1)
model2.load_state_dict(torch.load("five_crop.bin"))
model2.cuda()

for param in model2.parameters():
    param.requires_grad = False


In [16]:
def compare_models(model_1, model_2):
    models_differ = 0
    for key_item_1, key_item_2 in zip(model_1.state_dict().items(), model_2.state_dict().items()):
        if torch.equal(key_item_1[1], key_item_2[1]):
            pass
        else:
            models_differ += 1
            if (key_item_1[0] == key_item_2[0]):
                print('Mismtach found at', key_item_1[0])
            else:
                raise Exception
    if models_differ == 0:
        print('Models match perfectly! :)')

In [17]:
compare_models(model1, model2)

Mismtach found at features.conv0.weight
Mismtach found at features.norm0.weight
Mismtach found at features.norm0.bias
Mismtach found at features.norm0.running_mean
Mismtach found at features.norm0.running_var
Mismtach found at features.norm0.num_batches_tracked
Mismtach found at features.denseblock1.denselayer1.norm1.weight
Mismtach found at features.denseblock1.denselayer1.norm1.bias
Mismtach found at features.denseblock1.denselayer1.norm1.running_mean
Mismtach found at features.denseblock1.denselayer1.norm1.running_var
Mismtach found at features.denseblock1.denselayer1.norm1.num_batches_tracked
Mismtach found at features.denseblock1.denselayer1.conv1.weight
Mismtach found at features.denseblock1.denselayer1.norm2.weight
Mismtach found at features.denseblock1.denselayer1.norm2.bias
Mismtach found at features.denseblock1.denselayer1.norm2.running_mean
Mismtach found at features.denseblock1.denselayer1.norm2.running_var
Mismtach found at features.denseblock1.denselayer1.norm2.num_batche

Mismtach found at features.denseblock3.denselayer1.conv2.weight
Mismtach found at features.denseblock3.denselayer2.norm1.weight
Mismtach found at features.denseblock3.denselayer2.norm1.bias
Mismtach found at features.denseblock3.denselayer2.norm1.running_mean
Mismtach found at features.denseblock3.denselayer2.norm1.running_var
Mismtach found at features.denseblock3.denselayer2.norm1.num_batches_tracked
Mismtach found at features.denseblock3.denselayer2.conv1.weight
Mismtach found at features.denseblock3.denselayer2.norm2.weight
Mismtach found at features.denseblock3.denselayer2.norm2.bias
Mismtach found at features.denseblock3.denselayer2.norm2.running_mean
Mismtach found at features.denseblock3.denselayer2.norm2.running_var
Mismtach found at features.denseblock3.denselayer2.norm2.num_batches_tracked
Mismtach found at features.denseblock3.denselayer2.conv2.weight
Mismtach found at features.denseblock3.denselayer3.norm1.weight
Mismtach found at features.denseblock3.denselayer3.norm1.bia

In [28]:
model2 = models.densenet121(pretrained=True)
model2.classifier = nn.Linear(1024, 1)
model2.cuda()

model = models.densenet121(pretrained=True)
model.classifier = nn.Linear(1024, 1)
model.cuda()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [29]:
compare_models(model, model2)

Mismtach found at classifier.weight
Mismtach found at classifier.bias
